# **Word2Vec Tutorial**

* __Based on Lecture 2 from the Stanford Deep Learning Series by Christopher Manning__
* __Word2Vec: Set of algorithms developed by Mikolov et al. (2013)__
* __Original Word2Vec Paper: https://arxiv.org/abs/1301.3781__

### Question: How do we represent the meaning of a word computationally?
* Often we make use of a taxonomy like WordNet, using hypernym relationships and synonym sets.
* Problem with this:
    * Misses nuances (eg. 'good' and 'expert' are considered synonyms in WordNet, but "he's an expert at deep learning" and "he's good at deep learning" clearly have different meanings).
    * Misses new words (must constantly keep up to date), requires a lot of human labor.
    
    
* This can be considered a 'discrete' representation, using atomic symbols.
* Words like 'hotel', 'conference', 'walk', in vector space terms are represented using a vector of one 1 and many zeros [0 0 0 0 0 0 0 0 1 0 0 0], thus a 'one-hot' vector representation.
* These vectors end up being very long, depending on the vocabulary.
* Another problem: doesn't show any inherent relationship between words, eg. 'Seattle motel' and 'Seattle hotel' should be considered similar.

### Better approach:
* Find a way for the vectors to directly encode similarity &mdash; distributional similarity (the meaning of a word is the context it is in).
* "You shall know a word by the company it keeps." J. R. Firth 1957

* __Our goal:__ build a dense vector for each word, chosen so that it is good at predicting other words appearing in its context &mdash; the other words also represented by vectors of their own.
* Eg. linguistics = [0.286 0.792 -0.177 -0.107 0.109 -0.542 0.349 0.271]
* This type of dense vector representation is distributed &mdash; instead of an atomic 'one-hot' vector representation, the meaning of the word is 'smeared' over the whole vector with different numbers.

### Word2Vec &mdash; Set of algorithms developed by Tomas Mikolov in 2013

* These algorithms build dense vector representations for words, as outlined above.
* Became highly influential in statistical NLP.

### Basic idea:

* Define a model that aims to predict between a center word $w_t$ and context words in terms of word vectors:
    * $P(context \mid w_{t}) = \ldots$
    * Which has a loss function, eg.
    * $J = 1 - P(w_{-t} \mid w_{t})$ -- $w_{-t}$ refers to everything not $w_t$, hence all the words in the context
    * We look at many positions $t$ in a big language corpus.
    * We keep adjusting the vector representations of words to minimize this loss.
    * As a result, we obtain dense word vectors that are very powerful in representing meaning.

### Two algorithms of Word2Vec:

* __Skip-Gram (SG)__ &mdash; predicts context words given target
* __Continuous Bag of Words (CBOW)__ &mdash; predicts target word from bag-of-words context

### Two training methods:

* __Hierarchical softmax__
* __Negative sampling__

## Skip-Gram Model

<img src="SkipGramPrediction.png">

* For each word $t = 1 \ldots T$, predict surrounding words in a window of 'radius' $m$ of every word.
* Objective function: maximize the probability of any context word given the current center word.


* $\displaystyle J'(\theta) = \prod_{t=1}^T \prod_{\substack{-m \leq j \leq m\\ j \neq 0}} P(w_{t+j} \mid w_t ; \theta)$
* __Negative Log-Likelihood:__ $\displaystyle J(\theta) = -\frac{1}{T}\sum_{t=1}^T \sum_{\substack{-m \leq j \leq m\\ j \neq 0}} \log P(w_{t+j} \mid w_t)$
    * Where $\theta$ represents all variables we will optimize (the vector representations of the words)
    * Notes on second equation:
        * $-$ changes it to a minimization problem, since machine learning people like to minimize things, rather than maximizing them
        * $\frac{1}{T}$ &mdash; normalization, making it 'per word', instead of a probability for the whole corpus
        * $\log$ &mdash; our products turn into sums, making the math a lot easier

* __How is the probability itself measured?__


* For $P(w_{t+j} \mid w_t)$:
    * $P(o \mid c) = \frac{exp(u_{o}^T v_c)}{\sum_{w=1}^v exp(u_{w}^T v_c)}$ &mdash; _softmax_ form


* $o$: the outside/output word index
* $c$: the center word index
* $v_c$ and $u_o$: "center" and "outside" vectors of indices $c$ and $o$


* We put the dot product of the two vectors into a softmax form.
* The dot product will be bigger if $u$ and $v$ are more similar.
* Softmax form: standard way to turn numbers into a probability distribution.

    * $P_i = \frac{e^{u_i}}{\sum_{j}e^{u_j}}$


* Exponentiation makes sure everything is positive.
* Dividing by the sum of all the numbers gives a probability distribution.

* As a result of this function, each word has two representations: one for when it is the context word, another for when it is the center word.

## Final picture of the Skip-Gram model:


<img src="SkipGramDiagram.png">

## Training the model

$$\mathbf{\theta} = \left[\begin{array}
{r}
v_{aardvark} \\
v_{a} \\
\vdots \\
u_{aardvark} \\
u_{a} \\
\vdots \\
u_{zebra}
\end{array}\right]
\in {\rm I\!R}^{2dV}
$$

* $V$ &mdash; size of vocabulary
* $d$ &mdash; dimensionality of vector representations
* To train, compute the gradient for all vectors.

## Model Result

* The final word vectors obtained are able to capture some general and quite useful semantic information about words and their relationship to one another.
* In the latent vector space, different directions specialize towards different semantic and even grammatical relationships (male-female, verb tense, country-capital, and so on).
* This can be seen in the following graphs, using t-SNE for dimensionality reduction (from https://www.tensorflow.org/tutorials/word2vec):

<img src="W2VSemanticRelationships.png">

In addition, the following is a t-SNE visualization of learned Word2Vec embeddings, showing that similar words do indeed cluster nearby each other:

<img src="W2VVisualization.png">

# Building a Word2Vec Model

In [3]:
import gensim, logging, multiprocessing
import numpy as np
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
## Configuring the word2vec model
w2vconfig = {
    'min_count': 6, # minimum frequency of a word to be included in the model
    'hs': 0, # training method: 0 --- negative sampling, 1 --- hierarchical softmax
    'negative': 5, # number of "negative" words selected to update model weights
    'size': 100, # dimension of the vectors
    'sg': 0, # 0 --- CBOW, 1 --- skip-gram 
    'batch_words': 10000, # batch size for training the model
    'iter': 20, # number of epochs
    'window': 5, # context window
    'workers': multiprocessing.cpu_count(),
}

In [5]:
## Initializing the word2vec model using the above configuration
model = gensim.models.Word2Vec(**w2vconfig)

In [64]:
## Preparing the word2vec input (the gensim word2vec module expects a sequence of sentences as its input (list of lists))
# Eg. sentences = [['sentence', 'one'], ['sentence', 'two'], ...]
# Prior to making the model, helps to pre-process data (tokenization, stemming, POS-tagging, stop word/punctuation removal, etc.)

In [6]:
# Here the brown corpus will be used as a demonstration
from nltk.corpus import brown
sentences = brown.sents(categories=brown.categories())
print(len(sentences))
print(sentences[:2])

57340
[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']]


In [7]:
# Building the vocabulary and training the model
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)

2018-02-04 14:57:40,404 : INFO : collecting all words and their counts
2018-02-04 14:57:40,407 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-04 14:57:41,129 : INFO : PROGRESS: at sentence #10000, processed 219770 words, keeping 23488 word types
2018-02-04 14:57:41,881 : INFO : PROGRESS: at sentence #20000, processed 430477 words, keeping 34367 word types
2018-02-04 14:57:42,526 : INFO : PROGRESS: at sentence #30000, processed 669056 words, keeping 42365 word types
2018-02-04 14:57:43,181 : INFO : PROGRESS: at sentence #40000, processed 888291 words, keeping 49136 word types
2018-02-04 14:57:43,747 : INFO : PROGRESS: at sentence #50000, processed 1039920 words, keeping 53024 word types
2018-02-04 14:57:44,229 : INFO : collected 56057 word types from a corpus of 1161192 raw words and 57340 sentences
2018-02-04 14:57:44,231 : INFO : Loading a fresh vocabulary
2018-02-04 14:57:44,383 : INFO : min_count=6 retains 13160 unique words (23% of original 56057

2018-02-04 14:58:51,106 : INFO : PROGRESS: at 74.52% examples, 173429 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:52,108 : INFO : PROGRESS: at 75.72% examples, 173498 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:53,134 : INFO : PROGRESS: at 76.92% examples, 173782 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:54,145 : INFO : PROGRESS: at 78.05% examples, 174222 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:55,177 : INFO : PROGRESS: at 79.50% examples, 174319 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:56,186 : INFO : PROGRESS: at 80.75% examples, 174443 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:57,203 : INFO : PROGRESS: at 81.95% examples, 174721 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:58,239 : INFO : PROGRESS: at 83.08% examples, 175070 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:58:59,240 : INFO : PROGRESS: at 84.43% examples, 175040 words/s, in_qsize 0, out_qsize 0
2018-02-04 14:59:00,266 : INFO : PROGRESS: at 85.74% examples, 175203 wor

15409703

In [10]:
## Testing the model

print("Most similar words to 'person':", model.most_similar(positive=["person"]), "\n")
print("Most similar words to 'Asia':", model.most_similar(positive=["Asia"]), "\n")
print("Most similar words to 'two':", model.most_similar(positive=["two"]), "\n")
print("Most similar words to 'Fred':", model.most_similar(positive=["Fred"]), "\n")
print("'France' - 'Paris' + 'Germany' (expected 'Berlin'):", model.most_similar(positive=["France", "Germany"], negative=["Paris"]), "\n")

# Similar words:
print("Similarity of 'two' and 'three':", model.similarity("two", "three"), "\n")
print("Similarity of 'go' and 'come':", model.similarity("go", "come"), "\n")

# Dissimilar words:
print("Similarity of 'go' and 'orange':", model.similarity("go", "orange"))

Most similar words to 'person': [('child', 0.7117858529090881), ('man', 0.6841299533843994), ('woman', 0.6629840135574341), ('difficulty', 0.6074833869934082), ('teacher', 0.6068326830863953), ('patient', 0.602636456489563), ('artist', 0.5924439430236816), ('dancer', 0.589277446269989), ('word', 0.588111400604248), ('case', 0.5839000940322876)] 

Most similar words to 'Asia': [('Africa', 0.799575686454773), ('East', 0.772148847579956), ('South', 0.7577406167984009), ('Viet', 0.7552622556686401), ('Europe', 0.7377159595489502), ('Nam', 0.7342245578765869), ('North', 0.7217272520065308), ('Southeast', 0.7211916446685791), ('France', 0.714782178401947), ('Middle', 0.7034892439842224)] 

Most similar words to 'two': [('three', 0.8228723406791687), ('four', 0.7181538343429565), ('several', 0.6572548151016235), ('six', 0.6392922401428223), ('five', 0.6161919236183167), ('few', 0.5737988948822021), ('eight', 0.5627668499946594), ('Two', 0.5545819997787476), ('ten', 0.5273005962371826), ('seve

In [11]:
# Checking individual vectors
orange_vector = model.wv.syn0[model.wv.vocab["orange"].index]
print(orange_vector)

[ -2.56311476e-01   2.64052898e-01  -4.40935969e-01   9.09542441e-02
   1.93358675e-01  -3.28421444e-02  -7.56693408e-02  -8.06089044e-02
   4.26417142e-01   2.46043742e-01   4.28434104e-01   6.42067716e-02
  -1.90532133e-01  -2.35424474e-01  -2.47063249e-01  -7.06442177e-01
  -7.97080100e-02   6.57491386e-04  -3.12289327e-01  -2.54352927e-01
  -2.36614943e-01  -2.16081217e-01  -5.42365670e-01   3.05294245e-01
   1.70133665e-01  -2.63780534e-01   2.84948409e-01  -1.43995166e-01
   6.79376185e-01   3.22120003e-02   1.55920431e-01   2.93974906e-01
   1.10957816e-01  -2.55263507e-01  -3.12969148e-01  -4.03587192e-01
  -1.78760335e-01  -4.75222152e-03  -9.69010964e-02  -3.63178775e-02
  -1.15098897e-02  -2.23732904e-01  -4.28943157e-01   3.22065026e-01
   2.34043315e-01  -2.73598254e-01   8.57908744e-03   3.94227475e-01
  -2.97977962e-02  -1.71636477e-01   2.86382198e-01   5.98386228e-01
  -2.14478672e-01   8.30935389e-02  -3.59393835e-01  -1.28186807e-01
   1.67623907e-02   1.39775559e-01

* Alternatively, can use a pre-trained word2vec model.
* W2V model made by Google (1.5 GB, vocabulary of 3 million words, 100 billion words total): https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
* W2V models for many different languages: https://github.com/Kyubyong/wordvectors
* More W2V models: https://sites.google.com/site/rmyeid/projects/polyglot

* Example: loading the above word2vec model by Google
* large_model = gensim.models.KeyedVectors.load_word2vec_format('directory/GoogleNews-vectors-negative300.bin.gz', binary=True)